In [37]:
%run myCode.ipynb

In [36]:
dfAttacks.head(5)
#need to convert fatal into 1,0
dfM = dfAttacks.copy()
dfM['Fatal'] = dfM['Fatal'].apply(lambda x: 1 if x =='Y' else 0)
#dfM[['Gender','Fatal']]

In [120]:
dfCountry.to_csv("countryStatsX.csv", index=False)
dfSharks.to_csv("sharkStatsX.csv", index=False)
dfAttacks.to_csv("dfAttacksX.csv", index=False)

In [38]:
##The Imports
from cleantext import clean
from pandas.core.common import SettingWithCopyWarning
import warnings
import scipy as sc
import sklearn
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,TfidfTransformer
from sklearn.preprocessing import (
    OneHotEncoder, 
    OrdinalEncoder, 
    StandardScaler
)

import gensim
import gensim.downloader as model_api
word_vectors = model_api.load("glove-wiki-gigaword-50")
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objs as go
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
import lightgbm as lgb
import xgboost as xgb
###########################################################################################################################
#My NLP Helpers
####CLEANING THE TEXT all my helper functions 

def remove_html_tags(text):
    """Remove html tags from a string"""
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

def remove_emails(text):
    clean = re.compile("r'[\w\.-]+@[\w\.-]+'")
    return re.sub(clean, '', text)

def remove_emojis(text):
    return clean(text,no_emoji=True)

def remove_urls(text):
    return re.sub('http[s]?://\S+', '', text)
    
def finishCleaning(text):
    regex = re.compile(r"[0-9\/!@#$%^&*'()_+{}\[\]\-\:\.\,\?\"]+")
    return re.sub(regex,'', text)


def nlp_clean(text):
    s=remove_emojis(text)
    s=remove_html_tags(s)
    s=remove_emails(s)
    s=remove_urls(s)
    s=finishCleaning(s)
    s=s.lower()
    return s

def vectorizeParagraph(text): 
    text = nlp_clean(text)
    words = text.split()
    numwords = len(words)

    vector = np.zeros(50)
    myVectors = []
    
    for i in range (0, numwords):
        words[i] = re.sub(r'[^a-z]', '', str(words[i]))  ## This is to ensure there are no stray chars 
        if words[i] not in stop_words:
            try:
                index = vocab[words[i]]
                vector = word_vectors.get_vector(words[i])
                myVectors.append(vector)
            except:
                pass
   
    averageVector = np.mean(myVectors, axis = 0) #recommended way to make one vector ou of all the words in the paragraph
    return averageVector

#####This is much simpler to use than the function for GLOVE
def spacyVectorizeParagraph(text): 
    doc = nlp(text)

    return doc._.polarity
       

#words = pd.DataFrame(words.tolist())

In [39]:
#STRATEGY:
#WE WILL USE SPACY TOKENS ON THE 'Activity' COLUMN
#SHARK INFO IS NOT GOING TO BE IN THE COLUMNS HOWEVER A RISK FACTOR WILL BE DEPENDING ON COUNTRY AND LOCATION
#WE WILL USE RANDOMSEARCH OR GRIDSEARCH FOR HYPER PARAMETER TUNING
#SINCE THIS IS A BINARY CLASSIFICATION, I WANT TO LOOK INTO LGBM AND XGBOOST AND ALSO OUR TRUSTY LOGISTICAL REGRESSION
#!pip install lightgbm
#!pip install xgboost
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    f1_score,
    recall_score,
)
from sklearn.model_selection import (
    RandomizedSearchCV,
    GridSearchCV,
    cross_val_score,
    cross_validate,
    train_test_split,
)
from sklearn.preprocessing import (
    StandardScaler,
    PowerTransformer,
)

#from sklearn.preprocessing import PowerTransformer

from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
import xgboost as xgb
from sklearn.preprocessing import FunctionTransformer

from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
stop_words = set(ENGLISH_STOP_WORDS)
vocab = word_vectors.key_to_index

import spacy

In [40]:
lgbm = lgb.LGBMClassifier()
xgClassifier = xgb.XGBClassifier()

In [41]:
##LETS BEGIN!

#dfAttacks = pd.read_csv('dfAttacksX.csv')
#dfAttacks['Fatal']
#dfM = dfAttacks.copy()


In [42]:
def tokenize_text(text):
    return [tok.text for tok in nlp(text)]

In [43]:

##PREPROCESSOR AND COLUMN SELECTION AND TRANSFORMATION

nlp = spacy.load('en_core_web_sm')
categorical_features = ['Country', 'Gender','Timeslot','Zone']
numeric_features = ['latitude','longitude','risk']
#numeric_features = ['risk']
drop_features = ['CaseNumber', 'Date', 'Year', 'Type', 'Area', 'Location','Name', 'Injury','Time','Species', 'SharkSpecies', 'full_location', 'pos', 'latitude_rad','longitude_rad']
word_features = ['Activity']


train_df, test_df = train_test_split(dfM, test_size=0.2, random_state=5)

train_df.to_csv('train.csv', index=False)
test_df.to_csv('test.csv', index=False)

X_train = train_df.drop(columns='Fatal', axis=1)
y_train = train_df['Fatal']

X_test = test_df.drop(columns='Fatal', axis=1)
y_test = test_df['Fatal']



# Define the column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('Categorical', OneHotEncoder(handle_unknown='ignore'), categorical_features),
#        ('numerical', StandardScaler(), numeric_features),
        ('numerical', PowerTransformer(), numeric_features),
#        ('text', CountVectorizer(tokenizer=lambda text: [tok.text for tok in nlp(text)]), 'Activity'),
        ('text', TfidfVectorizer(tokenizer=tokenize_text), 'Activity'),
        ('droplist', 'drop', drop_features)
        ],
        remainder='passthrough'
    )


# Define the pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression())
])



# Fit the pipeline
pipeline.fit(X_train, y_train)

# Evaluate the pipeline
y_pred = pipeline.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

##Best Accuracy with Logistic Regression with PowerTransformer for numerical features and TFIDF word embedding
0.7696 
#Lets get RandomSearch or grid search into the mix with different classsifiers, I am happy with my preprocessor choices




0.7696

In [44]:
accuracy_score(y_train, pipeline.predict(X_train))

0.7894736842105263

In [45]:
##Now train the entire set
# Fit the pipeline
y_full = dfM['Fatal']
dfM = dfM.drop(columns='Fatal', axis=1)


pipeline.fit(dfM, y_full)

# Evaluate the pipeline
y_predfull = pipeline.predict(dfM)
accuracy = accuracy_score(y_full, y_predfull)

In [31]:
print(accuracy)

0.7925216598267214


In [46]:
with open('trained_pipeline_LogisticRegression.pkl', 'wb') as f:
    pickle.dump(pipeline, f)

In [47]:
with open('trained_pipeline_LogisticRegression.pkl', 'rb') as f:
    pipeline = pickle.load(f)


In [48]:
ytest_pred = pipeline.predict(X_test)
accuracy_score(y_test, ytest_pred)

0.8031914893617021

In [49]:
for i in range (0, len(y_predfull)):
    print(y_predfull[i])
    

N
N
Y
Y
Y
N
N
Y
Y
N
N
N
N
N
Y
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
Y
N
N
N
N
N
N
N
N
N
N
N
N
Y
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
Y
N
N
N
N
Y
N
N
N
N
N
N
Y
N
N
Y
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
Y
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
Y
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
Y
Y
N
Y
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
Y
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
Y
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
Y
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N


In [276]:
randomSearchLogistic.fit(X_train,y_train) 
print(randomSearchLogistic.best_params_)
randomSearchLogistic.best_score_

0.7476729532496326

In [278]:

print(randomSearchLogistic.best_params_)
print(randomSearchLogistic.best_score_)

{'classifier__solver': 'liblinear', 'classifier__penalty': 'l1', 'classifier__C': 5, 'classifier': LogisticRegression(C=5, penalty='l1', random_state=1, solver='liblinear')}
0.7476729532496326


In [234]:
#My Classifier this time is LGBM classifier

#My Hyper Param List
param_gridLoglgbm = {}
param_gridLoglgbm['classifier__max_depth'] = [3,5,7]
param_gridLoglgbm['classifier__learning_rate'] = [0.01,0.1,1]
param_gridLoglgbm['classifier__n_estimators'] = [100, 500, 1000]
param_gridLoglgbm['classifier__num_leaves'] = [10, 20, 30]
param_gridLoglgbm['classifier__min_child_samples'] = [10,20,30]
param_gridLoglgbm['classifier']=[lgbm]

lgbmclassifierparams = [param_gridLoglgbm]

#My Pipeline
pipelgbm = Pipeline([('preprocessor', preprocessor),('classifier',lgbm)])

# create GridSearchCV instance
random_search = RandomizedSearchCV(pipelgbm,lgbmclassifierparams, cv=3)

# fit GridSearchCV to data
random_search.fit(X_train, y_train)

# print best parameters and score
print("Best parameters found: ", grid_search.best_params_)
print("Best accuracy score found: ", grid_search.best_score_)

Best parameters found:  {'classifier__num_leaves': 20, 'classifier__n_estimators': 1000, 'classifier__min_child_samples': 20, 'classifier__max_depth': 7, 'classifier__learning_rate': 0.01, 'classifier': LGBMClassifier(learning_rate=0.01, max_depth=7, n_estimators=1000,
               num_leaves=20)}
Best accuracy score found:  0.7491916996668042


In [237]:
param_gridLoglgbm = {}
param_gridLoglgbm['classifier__max_depth'] = [10]
param_gridLoglgbm['classifier__learning_rate'] = [0.1]
param_gridLoglgbm['classifier__n_estimators'] = [1000]
param_gridLoglgbm['classifier__num_leaves'] = [20]
param_gridLoglgbm['classifier__min_child_samples'] = [20]
param_gridLoglgbm['classifier']=[lgbm]
# fit GridSearchCV to data
random_search.fit(X_train, y_train)

# print best parameters and score
print("Best parameters found: ", random_search.best_params_)
print("Best accuracy score found: ", random_search.best_score_)

# Best parameters found:  {'classifier__num_leaves': 30, 'classifier__n_estimators': 1000, 'classifier__min_child_samples': 10, 'classifier__max_depth': 5, 'classifier__learning_rate': 0.01, 'classifier': LGBMClassifier(learning_rate=0.01, max_depth=5, min_child_samples=10,
#                n_estimators=1000, num_leaves=30)}
# Best accuracy score found:  0.7450130538690022

Best parameters found:  {'classifier__num_leaves': 30, 'classifier__n_estimators': 1000, 'classifier__min_child_samples': 10, 'classifier__max_depth': 5, 'classifier__learning_rate': 0.01, 'classifier': LGBMClassifier(learning_rate=0.01, max_depth=5, min_child_samples=10,
               n_estimators=1000, num_leaves=30)}
Best accuracy score found:  0.7450130538690022


In [254]:
##TryingXGBoost but its beginning to look like 
#i am putting lipstick on a pig and this takes long, hoping to get an improvement above my 
#trusty logistic regression which still scores best so far

param_gridLogxgb = {}
param_gridLogxgb['classifier__Learning_rate'] = [0.03,0.3]
param_gridLogxgb['classifier__max_depth'] = [10, 15, 20]
param_gridLogxgb['classifier__gamma'] = [0.1, 0.2, 0.3, 0.4]
param_gridLogxgb['classifier__reg_alpha'] = [0, 0.001, 0.01]
param_gridLogxgb['classifier__scale_pos_weight'] = [1, 2, 5, 10]
param_gridLogxgb['classifier__n_estimators'] = [200, 400, 800]
param_gridLogxgb['classifier']=[xgClassifier]
xgbclassifierparams = [param_gridLogxgb]

#My Pipeline
pipexgb = Pipeline([('preprocessor', preprocessor),('classifier',xgClassifier)])

# create GridSearchCV instance
random_search2 = RandomizedSearchCV(pipexgb,xgbclassifierparams)

# fit GridSearchCV to data
random_search2.fit(X_train, y_train)

# print best parameters and score
print("Best parameters found: ", random_search2.best_params_)
print("Best accuracy score found: ", random_search2.best_score_)

[10:41:51] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "Learning_rate" } are not used.

[10:42:31] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "Learning_rate" } are not used.

[10:43:20] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "Learning_rate" } are not used.

[10:44:07] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "Learning_rate" } are not used.

[10:44:50] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "Learning_rate" } 

[11:13:48] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "Learning_rate" } are not used.

[11:14:30] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "Learning_rate" } are not used.

[11:15:10] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "Learning_rate" } are not used.

[11:15:51] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "Learning_rate" } are not used.

[11:16:30] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-07593ffd91cd9da33-1\xgboost\xgboost-ci-windows\src\learner.cc:767: 
Parameters: { "Learning_rate" } 

In [279]:
#xgClassifier

dfAttacks


,CaseNumber,Date,Year,Type,Country,Area,Location,Activity,Name,Gender,...,SharkSpecies,full_location,latitude,longitude,pos,Timeslot,Zone,latitude_rad,longitude_rad,risk
0,2023.03.19,3/19/2023,2023.0,Unprovoked,USA,Hawaii,Anaehoomalu Bay,Swimming,male,M,...,unknown,"Anaehoomalu Bay, Hawaii, USA",19.916124,-155.893224,"(19.9161242, -155.8932242)",Midday,TROPIC OF CANCER,0.347602,-2.720850,9.4
1,2023.03.16,3/16/2023,2023.0,Unprovoked,AUSTRALIA,Western Australia,Cheynes Beach,Surf-sking,female,F,...,unknown,"Cheynes Beach, Western Australia, AUSTRALIA",-34.787401,118.417953,"(-34.787401, 118.4179532)",Morning,POLAR,-0.607155,2.066783,10.0
2,2023.03.08,3/6/2023,2023.0,Unprovoked,BRAZIL,Pernambuco,Piedade Beach,Swimming,Kaylane Timóteo Freitas,F,...,unknown,"Piedade Beach, Pernambuco, BRAZIL",-8.167114,-34.913273,"(-8.1671139, -34.91327260000001)",Afternoon,TROPIC OF CAPRICORN,-0.142543,-0.609352,2.1
3,2023.03.05,3/5/2023,2023.0,Unprovoked,BRAZIL,Pernambuco,Piedade Beach,Swimming,male,M,...,unknown,"Piedade Beach, Pernambuco, BRAZIL",-8.167114,-34.913273,"(-8.1671139, -34.91327260000001)",Afternoon,TROPIC OF CAPRICORN,-0.142543,-0.609352,2.1
4,2023.03.02,3/2/2023,2023.0,Unprovoked,SEYCHELLES,Praslin Island,NaN,Snorkeling,Arthur …,M,...,lemon,"Praslin Island, SEYCHELLES",-4.332591,55.746726,"(-4.3325908, 55.74672589999999)",Afternoon,TROPIC OF CAPRICORN,-0.075618,0.972964,1.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6574,1742.12.17,1733-01-01,1733.0,Invalid,ICELAND,Bardestrand,Talkknefiord,swimming,NaN,M,...,unknown,"Talkknefiord, Bardestrand, ICELAND",64.963051,-19.020835,"(64.963051, -19.020835)",Afternoon,POLAR,1.133819,-0.331976,1.0
6575,1738.04.06.R,1723-01-01,1723.0,Unprovoked,ROATAN,NaN,NaN,swimming,Philip Ashton,M,...,unknown,", ROATAN",16.329761,-86.529967,"(16.3297608, -86.5299673)",Afternoon,TROPIC OF CANCER,0.285008,-1.510233,1.0
6576,1733.00.00,1721-06-01,1721.0,Unprovoked,ITALY,Sardinia,"Ponte della Maddelena,",Swimming,male,M,...,white shark,"Ponte della Maddelena,, Sardinia, ITALY",40.120875,9.012893,"(40.1208752, 9.012892599999999)",Afternoon,TEMPERATE,0.700241,0.157305,1.6
6577,1703.03.26,1703-03-26,1700.0,Unprovoked,FRANCE,Côte d'Azur,Antibes,Bathing,seaman,M,...,white shark,"Antibes, Côte d'Azur, FRANCE",43.580418,7.125102,"(43.58041799999999, 7.125102)",Afternoon,TEMPERATE,0.760622,0.124356,1.1
